# Astral tree on Amaranth-reference aligned reads

This notebook makes a species-level phylogeny for 38 Amaranth samples. <br>
Astral uses individual genes - here using tree_slider to determine the length - and estimates phylogeny in 2 step process. <br>
3RAD data assembled with ipyrad. modest filtering (see below) <br>
This is also the input for Nanuq, so output at bottom <br>

last run on Nov 12, 2020

In [1]:
# import packages into python
import ipyrad as ip
import ipyrad.analysis as ipa
import toytree
import toyplot.svg
import pandas as pd

# connect to parallel client
#import ipyparallel as ipp
#ipyclient = ipp.Client()
#ip.cluster_info(ipyclient)

# print the version of ipyrad you are running
print('ipyrad', ipa.__version__)
print('toytree', toytree.__version__)

ipyrad 0.9.61
toytree 2.0.4


First, use tree slider to get 1MB windows across the entire genome 

In [2]:
# the path to your HDF5 formatted seqs file
data = "/rigel/dsi/users/slh2181/tuberculatus_plate/ipyrad/Good_samp_beet_noMaxSNP_outfiles/Good_samp_beet_noMaxSNP.seqs.hdf5"
#set output directory
OUTDIR = "/rigel/dsi/users/slh2181/tuberculatus_plate/asplundii/hybridus_redo/vble_locus_length/"

In [3]:
# check scaffold idx (row) against scaffold names
ipa.treeslider(data).scaffold_table.head(20)

,scaffold_name,scaffold_length
0,Scaffold_10,22670516
1,Scaffold_11,22280117
2,Scaffold_12,22052327
3,Scaffold_13,20679869
4,Scaffold_14,20190685
5,Scaffold_15,17522127
6,Scaffold_16,16951160
7,Scaffold_1,38124660
8,Scaffold_2,35657244
9,Scaffold_3,30204323


In [4]:
imap={"acanthochiton": ["acanthochiton_SLH_AL_0001","acanthochiton_SLH_AL_0002"],
      "acutilobus": ["acutilobus_SLH_AL_0003","acutilobus_SLH_AL_0004"],
      "albus": ["albus_SLH_AL_0006", "albus_SLH_AL_0009", "albus_SLH_AL_0010"],
      "arenicola": ["arenicola_SLH_AL_0013","arenicola_SLH_AL_0018","arenicola_SLH_AL_0012","arenicola_SLH_AL_0015"],
 #     "asplundii": [],
      "australis": ["australis_SLH_AL_0020","australis_SLH_AL_0021"],
      "blitoides": ["blitoides_SLH_AL_0028"], #"blitoides_SLH_AL_0023",
      "blitum": ["blitum-blitum_SLH_AL_0029", "blitum-oleraceus_SLH_AL_0034", "blitum-pseudogracilis_SLH_AL_0037"],
      "californicus": ["californicus_SLH_AL_0039"],
      "cannabinus": ["cannabinus_SLH_AL_0040", "cannabinus_SLH_AL_0041"], #"cannabinus_SLH_AL_0042"],
      "caudatus": ["caudatus_SLH_AL_0102","caudatus_SLH_AL_0110","caudatus_SLH_AL_0116","caudatus_SLH_AL_0322","caudatus_SLH_AL_0540"],
      "crassipes": ["crassipes_SLH_AL_0599","crassipes_SLH_AL_0600"],
      "cruentus": ["cruentus_SLH_AL_0679", "cruentus_SLH_AL_0699", "cruentus_SLH_AL_0728", "cruentus_SLH_AL_0804", "cruentus_SLH_AL_0832"],
      "hybridus2": ["hybridus_SLH_AL_1060", "hybridus_SLH_AL_1098"],
      "deflexus": ["deflexus_SLH_AL_0951","deflexus_SLH_AL_0952", "deflexus_SLH_AL_0955","deflexus_SLH_AL_0953","deflexus_SLH_AL_0954"],
      "dubius": ["dubius_SLH_AL_0965","dubius_SLH_AL_0979","dubius_SLH_AL_0992"],
      "fimbriatus": [ "fimbriatus_SLH_AL_0998"], #"fimbriatus_SLH_AL_0997",
      "floridanus": ["floridanus_SLH_AL_1000"],
      "graecizans": ["graecizans-aschersonianus_SLH_AL_1009", "graecizans-silvestris_SLH_AL_1013", "graecizans-thellungianus_SLH_AL_1014", 
                    "asplundii_SLH_AL_0019"],
      "greggii": ["greggii_SLH_AL_1015", "greggii_SLH_AL_1016"],
      "hybridus1": ["hybridus_SLH_AL_0001-restricted", "hybridus_SLH_AL_1117"],
      "hybridus3": ["hybridus_SLH_AL_1099"], 
      "hypochondriacus": ["hypochondriacus_SLH_AL_1178", "hypochondriacus_SLH_AL_1197", "hypochondriacus_SLH_AL_1264", "hypochondriacus_SLH_AL_1285","hypochondriacus_SLH_AL_2282", "hypochondriacus_SLH_AL_2436"],
      "muricatus": ["muricatus_SLH_AL_2634"],
      "palmeri": ["palmeri-aff_SLH_AL_0017", "palmeri_SLH_AL_2637", "palmeri_SLH_AL_2644", "palmeri_SLH_AL_2647", "palmeri_SLH_AL_2649", "palmeri_SLH_AL_2650",
                  "palmeri_SLH_AL_2635","palmeri_SLH_AL_2636","palmeri_SLH_AL_2638","palmeri_SLH_AL_2639","palmeri_SLH_AL_2640","palmeri_SLH_AL_2641","palmeri_SLH_AL_2642","palmeri_SLH_AL_2643","palmeri_SLH_AL_2645","palmeri_SLH_AL_2646",
                  "palmeri_SLH_AL_159-contemp","palmeri_SLH_AL_163-contemp","palmeri_SLH_AL_173-contemp","palmeri_SLH_AL_174-contemp","palmeri_SLH_AL_235-contemp"],
      "powellii": ["powellii-bouchonii_SLH_AL_2653", "powellii-powellii_SLH_AL_2663", "powellii-powellii_SLH_AL_2665"],
      "pumilus": ["pumilus_SLH_AL_5-restricted","pumilus_SLH_AL_7-restricted"],
      "quitensis": ["quitensis_SLH_AL_2671", "quitensis_SLH_AL_2675","quitensis_SLH_AL_2753"],
      "retroflexus": ["retroflexus_SLH_AL_2770", "retroflexus_SLH_AL_2773", "retroflexus_SLH_AL_2780"],
      "spinosus": ["spinosus_SLH_AL_2792", "spinosus_SLH_AL_2793", "spinosus_SLH_AL_2806", "spinosus_SLH_AL_2809", "spinosus_SLH_AL_2811"],
      "standleyanus": ["standleyanus_SLH_AL_2815","standleyanus_SLH_AL_2816"],
      "tamaulipensis": ["tamaulipensis_SLH_AL_2817"],
      "torreyi": ["torreyi_SLH_AL_2818"],
      "tricolor": ["tricolor_SLH_AL_2869", "tricolor_SLH_AL_2940", "tricolor_SLH_AL_2953", "tricolor_SLH_AL_2978"],
      "tuberculatus": ["tuberculatus_SLH_AL_0009-restricted", "tuberculatus_SLH_AL_3003", "tuberculatus_SLH_AL_3017", "tuberculatus_SLH_AL_3027", "tuberculatus_SLH_AL_3045",
                        "tuberculatus_SLH_AL_2999","tuberculatus_SLH_AL_3000","tuberculatus_SLH_AL_3001","tuberculatus_SLH_AL_3002","tuberculatus_SLH_AL_3004","tuberculatus_SLH_AL_3005",
                        "tuberculatus_SLH_AL_3006","tuberculatus_SLH_AL_3007","tuberculatus_SLH_AL_3008","tuberculatus_SLH_AL_3009","tuberculatus_SLH_AL_3010",
                        "tuberculatus_SLH_AL_3011","tuberculatus_SLH_AL_3012","tuberculatus_SLH_AL_3013","tuberculatus_SLH_AL_3014","tuberculatus_SLH_AL_3015",
                        "tuberculatus_SLH_AL_3016","tuberculatus_SLH_AL_3018","tuberculatus_SLH_AL_3019","tuberculatus_SLH_AL_3020","tuberculatus_SLH_AL_3021",
                        "tuberculatus_SLH_AL_3022","tuberculatus_SLH_AL_3023","tuberculatus_SLH_AL_3024","tuberculatus_SLH_AL_3025","tuberculatus_SLH_AL_3026",
                        "tuberculatus_SLH_AL_3028","tuberculatus_SLH_AL_3029","tuberculatus_SLH_AL_3030","tuberculatus_SLH_AL_3031","tuberculatus_SLH_AL_3032",
                        "tuberculatus_SLH_AL_3033","tuberculatus_SLH_AL_3034","tuberculatus_SLH_AL_3035","tuberculatus_SLH_AL_3036",#"tuberculatus_SLH_AL_3037",
                        "tuberculatus_SLH_AL_3038","tuberculatus_SLH_AL_3039","tuberculatus_SLH_AL_3041","tuberculatus_SLH_AL_3042","tuberculatus_SLH_AL_3043",
                        "tuberculatus_SLH_AL_3044","tuberculatus_SLH_AL_3046","tuberculatus_SLH_AL_154-contemp","tuberculatus_SLH_AL_155-contemp",
                        "tuberculatus_SLH_AL_156-contemp","tuberculatus_SLH_AL_157-contemp","tuberculatus_SLH_AL_160-contemp","tuberculatus_SLH_AL_165-contemp",
                        "tuberculatus_SLH_AL_169-contemp","tuberculatus_SLH_AL_175-contemp","tuberculatus_SLH_AL_176-contemp","tuberculatus_SLH_AL_202-contemp",
                        "tuberculatus_SLH_AL_206-contemp","tuberculatus_SLH_AL_208-contemp","tuberculatus_SLH_AL_236-contemp","tuberculatus_SLH_AL_237-contemp"],
        "tucsonensis": ["tucsonensis_SLH_AL_3068"],
        "viridis": ["viridis_SLH_AL_3047", "viridis_SLH_AL_3062"],
        "watsonii": ["watsonii_SLH_AL_3065"],
        "wrightii": ["wrightii_SLH_AL_3066", "wrightii_SLH_AL_3067"],    
      "beet":["beet"],
     }

### Astral does not use bootstrap on gene trees created with RAxML, so don't bother running bootstraps

In [5]:
#run RAxML on "genes" that are 1M bp long in treeslider
ts = ipa.treeslider(
        name='tub_cons_noMaxSNP_1Mb_100SNP',  #this name can't end in _2
        data=data,
        workdir=OUTDIR,
        scaffold_idxs=range(16),
        window_size=1000000,
        slide_size= 1000000,  
        inference_method="raxml",  #options are raxml and mrbayes
        inference_args={"N": 1, "T": 24}, #number of bootstraps and # cores
        consensus_reduce=True, # this tells it to make 1 sequence for each key in dictionary
        minsnps=100,  #within window_size, it must have at least this much diversity
        mincov=4,   #For example, mincov=0.5 will require that 50% of samples contain a site that is not N or - for the site to be included in the alignment. 
        rmincov=0,
        imap=imap,
        minmap= {i: 0.1 for i in imap}, #half of each key in imap dictionary
  #      keep_all_files=True,
)

In [6]:
#check that the names I specificed in IMAP are the names in the dataset.
# 1. Print names in dataset
goodnames = set(ts._pnames)
goodnames

{'acanthochiton_SLH_AL_0001',
 'acanthochiton_SLH_AL_0002',
 'acutilobus_SLH_AL_0003',
 'acutilobus_SLH_AL_0004',
 'albus_SLH_AL_0006',
 'albus_SLH_AL_0009',
 'albus_SLH_AL_0010',
 'arenicola_SLH_AL_0012',
 'arenicola_SLH_AL_0013',
 'arenicola_SLH_AL_0015',
 'arenicola_SLH_AL_0018',
 'asplundii_SLH_AL_0019',
 'australis_SLH_AL_0020',
 'australis_SLH_AL_0021',
 'beet',
 'blitoides_SLH_AL_0023',
 'blitoides_SLH_AL_0028',
 'blitum-blitum_SLH_AL_0029',
 'blitum-oleraceus_SLH_AL_0034',
 'blitum-pseudogracilis_SLH_AL_0037',
 'californicus_SLH_AL_0039',
 'cannabinus_SLH_AL_0040',
 'cannabinus_SLH_AL_0041',
 'caudatus_SLH_AL_0102',
 'caudatus_SLH_AL_0110',
 'caudatus_SLH_AL_0116',
 'caudatus_SLH_AL_0322',
 'caudatus_SLH_AL_0540',
 'crassipes_SLH_AL_0599',
 'crassipes_SLH_AL_0600',
 'cruentus_SLH_AL_0679',
 'cruentus_SLH_AL_0699',
 'cruentus_SLH_AL_0728',
 'cruentus_SLH_AL_0804',
 'cruentus_SLH_AL_0832',
 'deflexus_SLH_AL_0951',
 'deflexus_SLH_AL_0952',
 'deflexus_SLH_AL_0953',
 'deflexus_SLH_A

In [7]:
# 2. Print the names that have typos
for key, vals in ts.imap.items():
    diff = set(vals).difference(goodnames)
    print(diff)

set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()


In [13]:
# run treeslider/RAxML.  this takes a long time (8hrs)
# make sure there is no temp directory existing because then this won't run.
ts.run(auto=True, force=True) #use auto=true when you have 24 engines

building database: nwindows=388; minsnps=100
[####################] 100% 0:18:56 | inferring trees 
tree_table written to /rigel/dsi/users/slh2181/tuberculatus_plate/asplundii/hybridus_redo/vble_locus_length/tub_cons_noMaxSNP_1Mb_100SNP.tree_table.csv


In [14]:
ts.tree_table.tail(10)

,scaffold,start,end,sites,snps,samples,missing,tree
378,15,12000000,13000000,9070,627,39,0.49,None
379,15,13000000,14000000,7173,620,39,0.49,"(greggii:0.00630941,acanthochiton:0.0107961,(arenicola:0.0111434,(pumilus:0.0132128,((floridanus:0.00955239,(beet:0.225107,tuberculatus:0.00196708)0:0.00796614)0:0.00646862,((australis:0.00749676,cannabinus:0.00706899)0:0.0135817,(((torreyi:0.013..."
380,15,14000000,15000000,13246,990,39,0.49,"(acanthochiton:0.00942685,(arenicola:0.00721672,greggii:0.00717856)0:0.00296792,(pumilus:0.0135523,((floridanus:0.00950584,tuberculatus:0.0110098)0:0.00304553,((australis:0.00567616,cannabinus:0.00860449)0:0.00898708,(((tamaulipensis:0.00900067,c..."
381,15,15000000,16000000,12750,849,39,0.50,None
382,15,16000000,17000000,12845,924,39,0.46,"(acanthochiton:0.0128421,(arenicola:0.00720738,greggii:0.00511216)0:0.000960657,(pumilus:0.015095,(tuberculatus:0.0108891,(floridanus:0.00742048,((australis:0.00738648,cannabinus:0.00896505)0:0.00704293,(((dubius:0.00932507,(palmeri:0.0136678,(sp..."
383,15,17000000,18000000,12130,910,39,0.43,"(pumilus:0.00909386,acanthochiton:0.00947063,(greggii:0.00886545,((beet:0.203868,arenicola:0.00634526)0:0.00242485,((floridanus:0.00916723,(tuberculatus:0.0125622,(australis:0.00626066,cannabinus:0.0047345)0:0.0110357)0:0.00171647)0:0.00206212,((..."
384,15,18000000,19000000,9930,692,39,0.42,"(pumilus:0.0134243,acanthochiton:0.0086638,((tuberculatus:0.0105999,(greggii:0.00587098,arenicola:0.00829999)0:0.00302599)0:0.00345437,(floridanus:0.00963465,((australis:0.00626347,cannabinus:0.00571386)0:0.00702233,(((torreyi:0.00948642,(blitoid..."
385,15,19000000,20000000,10795,847,39,0.46,"(acanthochiton:0.014947,(pumilus:0.0118707,(arenicola:0.0136109,(tuberculatus:0.0132828,greggii:0.0065918)0:0.00168878)0:0.00309734)0:0.00177003,((floridanus:0.00823478,(australis:0.00703142,cannabinus:0.00420057)0:0.00546237)0:0.00535068,((((bee..."
386,15,20000000,21000000,6432,489,39,0.40,"(arenicola:0.00537723,acanthochiton:0.00902722,((pumilus:0.0100254,greggii:0.00691355)0:0.00190801,((floridanus:0.00560375,tuberculatus:0.00712264)0:0.00299523,((australis:0.00358915,cannabinus:0.00387308)0:0.00816063,((torreyi:0.0162438,((crassi..."
387,15,21000000,22000000,9729,703,39,0.45,"(pumilus:0.00870676,acanthochiton:0.0113906,(greggii:0.00914813,((arenicola:0.00887791,floridanus:0.0123182)0:0.00220557,(tuberculatus:0.0133402,((australis:0.00720857,cannabinus:0.00811631)0:0.00811164,(((torreyi:0.00982336,((tamaulipensis:0.007..."


Check out the Treeslider results

In [15]:
ts.tree_table.describe()

,scaffold,start,end,sites,snps,samples,missing
count,388.000,3.880e+02,3.880e+02,388.000,388.000,388.0,388.000
mean,7.758,1.231e+07,1.331e+07,7623.753,554.072,39.0,0.517
std,4.428,8.231e+06,8.231e+06,3763.000,285.607,0.0,0.087
min,0.000,0.000e+00,1.000e+06,1412.000,62.000,39.0,0.320
25%,4.000,6.000e+06,7.000e+06,4223.500,300.750,39.0,0.450
50%,8.000,1.200e+07,1.300e+07,7166.500,519.500,39.0,0.500
75%,11.000,1.800e+07,1.900e+07,10503.000,778.750,39.0,0.570
max,15.000,3.700e+07,3.800e+07,23052.000,1349.000,39.0,0.780


In [16]:
Ast = ipa.astral(
    data = "/rigel/dsi/users/slh2181/tuberculatus_plate/asplundii/hybridus_redo/vble_locus_length/tub_cons_noMaxSNP_1Mb_100SNP.tree_table.csv",
    name='Astral_asplundii_fixed',
    workdir=OUTDIR,
   # bootsfile="/rigel/dsi/users/slh2181/tuberculatus_plate/Analysis/Consensus_tree3/tub_cons_noMaxSNP_1Mb_100SNP.bootsfiles.txt",
    imap=None,
    annotation=1,
   # gene_resampling=True,
   # nboots=100,
)

Ast.print_command()

java -jar /rigel/home/slh2181/miniconda3/bin/astral.5.7.1.jar -i /rigel/dsi/users/slh2181/tuberculatus_plate/asplundii/hybridus_redo/vble_locus_length/tmptrees.txt -o /rigel/dsi/users/slh2181/tuberculatus_plate/asplundii/hybridus_redo/vble_locus_length/Astral_asplundii_fixed.tre -t 1


In [17]:
Ast.run()

[astral.5.7.1.jar]
inferred tree written to (/rigel/dsi/users/slh2181/tuberculatus_plate/asplundii/hybridus_redo/vble_locus_length/Astral_asplundii_fixed.tre)


# Draw the Astral Tree

In [18]:
rooted1= Ast.tree.root("beet") #root on entire orange clade???
our_labels = rooted1.get_tip_labels()

color_dict = {
"quitensis": "orange",# 'quitensis',
"caudatus": "orange",# 'caudatus',
"hypochondriacus": "orange",# 'hypochondriacus',
"reference": "orange",
"hybridus1": "orange",# 'hybridus',
"hybridus2": "orange",# 'hybridus',
"hybridus3": "orange",# 'hybridus',
"cruentus": "orange",# 'cruentus',
"wrightii": "orange",# 'wrightii',
"retroflexus": "orange",# 'retroflexus',
"powellii": "orange",# 'powellii',
"acutilobus": "orange",# 'acutilobus',
"watsonii": "orange",# 'watsonii',
"palmeri": "orange",# 'palmeri',
"spinosus": "orange",# 'spinosus',
"dubius": "orange",# 'dubius',
"arenicola": "blue",# 'arenicola',
"greggii": "blue",# 'greggii',
"acanthochiton": "blue",# 'acanthochiton',
"pumilus": "blue",# 'pumilus',
"floridanus": "blue",# 'floridanus',
"tuberculatus": "blue",# 'tuberculatus',
"cannabinus": "blue",# 'cannabinus',
"australis": "blue",# 'australis',
"viridis": "red",# 'viridis',
"deflexus": "red",# 'deflexus',
"muricatus": "red",# 'muricatus',
"standleyanus": "red",# 'standleyanus',
"asplundii": "purple",# 'asplundii',
"graecizans": "purple",# 'graecizans',
"tricolor": "purple",# 'tricolor',
"blitum": "purple",# 'blitum',
"albus": "green",# 'albus',
"californicus": "green",# 'californicus',
"blitoides": "green",# 'blitoides',
"torreyi": "green",# 'torreyi',
"crassipes": "green",# 'crassipes',
"tamaulipensis": "green",# 'tamaulipensis',
"fimbriatus": "green",# 'fimbriatus',
"tucsonensis": "green",# 'tucsonensis',
"beet": "black",# 'beet'
"unknown": "black",
}

sp_names= [i.split("_")[0].split("-")[0] for i in our_labels]
colors = [color_dict[sp] for sp in sp_names]

In [19]:
rooted1.treenode.support = 100
for node in rooted1.treenode.traverse():
    node.support = int(float(node.support))
    
canvas, axes, mark=rooted1.draw(tip_labels_align=True, node_labels="support", 
                          use_edge_lengths=False,                        
                          node_labels_style={"font-size": "15px"},
                        node_sizes=27,
                        node_style={
                                "fill": "white",
                                "stroke": "black",
                        },
                        tip_labels_colors=colors,
                        tip_labels_style={"font-size":"16px"},
                        width=600, height=800);

<svg class="toyplot-canvas-Canvas" height="800.0px" id="tf162faff29b24885ae063c2a94d05841" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 600.0 800.0" width="600.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 70 77 90 86 85 97 46 57 49 90 52 79 52 45 89 97 48 62 60 42 90 69 53 79 71 66 98 88 61 83 63 96 96 51 38 68 100 viridis deflexus muricatus standleyanus graecizans tricolor blitum californicus albus blitoides torreyi crassipes tamaulipensis fimbriatus tucsonensis arenicola greggii acanthochiton pumilus tuberculatus floridanus cannabinus australis quitensis caudatus hybridus1 hypochondriacus hybridus3 cruentus hybridus2 retroflexus wrightii powellii acutilobus watsonii palmeri spinosus dubius beet

In [20]:
toyplot.svg.render(canvas,"/rigel/dsi/users/slh2181/tuberculatus_plate/asplundii/hybridus_redo/vble_locus_length/Astral_asplundii_12nov2020.svg")

# Save the tree.csv for Nanuq input

In [21]:
#tree_table = pd.read_csv("/rigel/dsi/users/slh2181/tuberculatus_plate/Analysis/Consensus_tree3/tub_cons_noMaxSNP_1Mb_75SNP.tree_table.csv", sep=",")
new = ts.tree_table.tree[ts.tree_table.tree.notna()].reset_index(drop=True)
new.to_csv(
    "/rigel/dsi/users/slh2181/tuberculatus_plate/asplundii/hybridus_redo/vble_locus_length/tub_cons_noMaxSNP_1Mb_100SNP.tree.csv",
    header=False, encoding='utf-8', index=False, sep=" ")